### Transfer learning in action
##### Check that we have correct libraries installed

In [2]:
import tensorflow
import keras
import h5py
assert(tensorflow.__version__ == '1.12.0')
assert(keras.__version__ == '2.2.4')
assert(h5py.__version__ == '2.7.0')

/databricks/python/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
 from ._conv import register_converters as _register_converters
Using TensorFlow backend.

##### Download the flower dataset

In [4]:
%sh
curl -O http://download.tensorflow.org/example_images/flower_photos.tgz
tar xzf flower_photos.tgz &>/dev/null
cd flower_photos
ls
pwd

% Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed

 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
 14 218M 14 32.0M 0 0 21.3M 0 0:00:10 0:00:01 0:00:09 21.3M
 36 218M 36 80.0M 0 0 36.9M 0 0:00:05 0:00:02 0:00:03 36.9M
 65 218M 65 144M 0 0 45.0M 0 0:00:04 0:00:03 0:00:01 45.0M
 95 218M 95 208M 0 0 46.6M 0 0:00:04 0:00:04 --:--:-- 46.6M
100 218M 100 218M 0 0 48.5M 0 0:00:04 0:00:04 --:--:-- 50.0M
daisy
dandelion
LICENSE.txt
roses
sunflowers
tulips
/databricks/driver/flower_photos

In [5]:
img_dir = 'file:/databricks/driver/flower_photos'

##### Load images into Dataframe

##### Using Spark's ImageSchema

In [8]:
from pyspark.ml.image import ImageSchema
df = ImageSchema.readImages(img_dir)
df.show()
df.printSchema()

+--------------------+
 image|
+--------------------+
[file:/databricks...|
+--------------------+

root
-- image: struct (nullable = true)
 |-- origin: string (nullable = true)
 |-- height: integer (nullable = false)
 |-- width: integer (nullable = false)
 |-- nChannels: integer (nullable = false)
 |-- mode: integer (nullable = false)
 |-- data: binary (nullable = false)

##### Using custom image library

In [10]:
from sparkdl.image import imageIO
idf = imageIO.readImagesWithCustomFn(img_dir, decode_f=imageIO.PIL_decode)
idf.show()
idf.printSchema()

+-----+
image|
+-----+
 null|
+-----+

root
-- image: struct (nullable = true)
 |-- origin: string (nullable = true)
 |-- height: integer (nullable = false)
 |-- width: integer (nullable = false)
 |-- nChannels: integer (nullable = false)
 |-- mode: integer (nullable = false)
 |-- data: binary (nullable = false)

##### Create the custom classifier

In [13]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from sparkdl import DeepImageFeaturizer

In [14]:
featurizer = DeepImageFeaturizer(inputCol='image', outputCol='features',modelName='ResNet50')

In [15]:
lr = LogisticRegression(maxIter=10, regParam=0.05, elasticNetParam=0.3, labelCol='label')

In [16]:
flo = Pipeline(stages=[featurizer, lr])

##### Create the train and test class

In [19]:
from pyspark.ml.image import ImageSchema
from pyspark.sql.functions import lit
from sparkdl.image import imageIO

In [20]:
tulips_df = ImageSchema.readImages(img_dir + '/tulips').withColumn('label',lit(0))
sun_df = imageIO.readImagesWithCustomFn(img_dir + '/sunflowers',decode_f=imageIO.PIL_decode).withColumn('label',lit(1))
tulips_train, tulips_test, _ = tulips_df.randomSplit([0.08, 0.08, 0.84])
sun_train, sun_test, _ = sun_df.randomSplit([0.08, 0.08, 0.84])
train_df = tulips_train.unionAll(sun_train)
test_df = tulips_test.unionAll(sun_test)
train_df = train_df.repartition(100)
test_df = test_df.repartition(100)

##### Train the model

In [23]:
f_model = flo.fit(train_df)

##### Test the accuracy

In [26]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [27]:
tested_df = f_model.transform(test_df)
evaluator = MulticlassClassificationEvaluator(metricName='accuracy')
print('Test set accuracy : ' + str(evaluator.evaluate(tested_df.select('prediction','label'))))

Test set accuracy : 0.9016393442622951

##### Using a test image